In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 128
#model_name = "VGG16_quant"
#model = VGG16_quant()
model_name = "resnet20_quant"
model = resnet20_quant()
print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    #adjust_list = [150, 225]
    adjust_list = [80, 120]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
ResNet_Cifar(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()

In [2]:
model.conv1 = QuantConv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
print(model.conv1)

for layer in model.modules():
    if isinstance(layer, torch.nn.Conv2d):
        layer.bit = 2
        layer.act_alpha = torch.nn.Parameter(torch.tensor(4.0))

QuantConv2d(
  3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)


In [14]:
print(model)

ResNet_Cifar(
  (conv1): QuantConv2d(
    3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
    (weight_quant): weight_quantize_fn()
  )
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (we

In [4]:
# This cell won't be given, but students will complete the training

lr = 4e-2
weight_decay = 1e-4
epochs = 100
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name) + "_2bit"
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: [0][0/391]	Time 0.604 (0.604)	Data 0.473 (0.473)	Loss 2.4749 (2.4749)	Prec 11.719% (11.719%)
Epoch: [0][100/391]	Time 0.052 (0.055)	Data 0.002 (0.007)	Loss 1.7646 (1.9481)	Prec 32.812% (26.261%)
Epoch: [0][200/391]	Time 0.058 (0.051)	Data 0.003 (0.005)	Loss 1.5181 (1.7958)	Prec 45.312% (32.424%)
Epoch: [0][300/391]	Time 0.051 (0.050)	Data 0.003 (0.004)	Loss 1.4983 (1.6921)	Prec 49.219% (36.836%)
Validation starts
Test: [0/79]	Time 0.344 (0.344)	Loss 1.4855 (1.4855)	Prec 42.969% (42.969%)
 * Prec 48.730% 
best acc: 48.730000
Epoch: [1][0/391]	Time 0.739 (0.739)	Data 0.682 (0.682)	Loss 1.2946 (1.2946)	Prec 50.000% (50.000%)
Epoch: [1][100/391]	Time 0.045 (0.053)	Data 0.002 (0.009)	Loss 1.2185 (1.2413)	Prec 58.594% (55.585%)
Epoch: [1][200/391]	Time 0.053 (0.050)	Data 0.002 (0.006)	Loss 1.0442 (1.2019)	Prec 63.281% (57.156%)
Epoch: [1][300/391]	Time 0.047 (0.049)	Data 0.002 (0.004)	Loss 1.0309 (1.1710)	Prec 64.062% (58.127%)
Validation starts
Test: [0/79]	Time 0.481 (0.481)	Loss 0.

Epoch: [15][200/391]	Time 0.040 (0.048)	Data 0.002 (0.004)	Loss 0.3820 (0.4340)	Prec 85.156% (84.888%)
Epoch: [15][300/391]	Time 0.044 (0.046)	Data 0.001 (0.003)	Loss 0.3717 (0.4330)	Prec 88.281% (84.969%)
Validation starts
Test: [0/79]	Time 0.728 (0.728)	Loss 0.5524 (0.5524)	Prec 82.812% (82.812%)
 * Prec 82.460% 
best acc: 82.460000
Epoch: [16][0/391]	Time 0.609 (0.609)	Data 0.541 (0.541)	Loss 0.3919 (0.3919)	Prec 85.938% (85.938%)
Epoch: [16][100/391]	Time 0.053 (0.052)	Data 0.002 (0.007)	Loss 0.3739 (0.4152)	Prec 87.500% (85.558%)
Epoch: [16][200/391]	Time 0.045 (0.049)	Data 0.002 (0.004)	Loss 0.4845 (0.4141)	Prec 83.594% (85.560%)
Epoch: [16][300/391]	Time 0.042 (0.047)	Data 0.001 (0.004)	Loss 0.6894 (0.4179)	Prec 78.906% (85.437%)
Validation starts
Test: [0/79]	Time 0.335 (0.335)	Loss 0.5601 (0.5601)	Prec 78.125% (78.125%)
 * Prec 80.520% 
best acc: 82.460000
Epoch: [17][0/391]	Time 0.715 (0.715)	Data 0.647 (0.647)	Loss 0.3407 (0.3407)	Prec 87.500% (87.500%)
Epoch: [17][100/391]	

Epoch: [30][300/391]	Time 0.042 (0.048)	Data 0.002 (0.004)	Loss 0.3388 (0.3048)	Prec 90.625% (89.371%)
Validation starts
Test: [0/79]	Time 0.499 (0.499)	Loss 0.3089 (0.3089)	Prec 88.281% (88.281%)
 * Prec 84.410% 
best acc: 84.660000
Epoch: [31][0/391]	Time 0.561 (0.561)	Data 0.471 (0.471)	Loss 0.3385 (0.3385)	Prec 89.062% (89.062%)
Epoch: [31][100/391]	Time 0.048 (0.052)	Data 0.002 (0.007)	Loss 0.2508 (0.2995)	Prec 91.406% (89.836%)
Epoch: [31][200/391]	Time 0.050 (0.051)	Data 0.003 (0.005)	Loss 0.2211 (0.3037)	Prec 92.969% (89.486%)
Epoch: [31][300/391]	Time 0.040 (0.050)	Data 0.002 (0.004)	Loss 0.2368 (0.3083)	Prec 91.406% (89.231%)
Validation starts
Test: [0/79]	Time 0.345 (0.345)	Loss 0.4305 (0.4305)	Prec 86.719% (86.719%)
 * Prec 83.760% 
best acc: 84.660000
Epoch: [32][0/391]	Time 0.509 (0.509)	Data 0.446 (0.446)	Loss 0.2512 (0.2512)	Prec 91.406% (91.406%)
Epoch: [32][100/391]	Time 0.044 (0.049)	Data 0.002 (0.006)	Loss 0.2219 (0.2965)	Prec 91.406% (89.519%)
Epoch: [32][200/391]	

Validation starts
Test: [0/79]	Time 0.356 (0.356)	Loss 0.3609 (0.3609)	Prec 85.938% (85.938%)
 * Prec 85.380% 
best acc: 87.010000
Epoch: [46][0/391]	Time 0.469 (0.469)	Data 0.374 (0.374)	Loss 0.1617 (0.1617)	Prec 93.750% (93.750%)
Epoch: [46][100/391]	Time 0.052 (0.052)	Data 0.002 (0.006)	Loss 0.1948 (0.2436)	Prec 92.188% (91.615%)
Epoch: [46][200/391]	Time 0.045 (0.049)	Data 0.002 (0.004)	Loss 0.3070 (0.2408)	Prec 87.500% (91.585%)
Epoch: [46][300/391]	Time 0.045 (0.046)	Data 0.002 (0.003)	Loss 0.2141 (0.2501)	Prec 92.969% (91.243%)
Validation starts
Test: [0/79]	Time 0.327 (0.327)	Loss 0.3609 (0.3609)	Prec 87.500% (87.500%)
 * Prec 85.470% 
best acc: 87.010000
Epoch: [47][0/391]	Time 0.563 (0.563)	Data 0.510 (0.510)	Loss 0.1964 (0.1964)	Prec 92.969% (92.969%)
Epoch: [47][100/391]	Time 0.046 (0.050)	Data 0.002 (0.007)	Loss 0.2707 (0.2363)	Prec 89.844% (91.692%)
Epoch: [47][200/391]	Time 0.045 (0.048)	Data 0.002 (0.004)	Loss 0.2359 (0.2380)	Prec 92.969% (91.647%)
Epoch: [47][300/391]	

 * Prec 86.000% 
best acc: 87.060000
Epoch: [61][0/391]	Time 0.624 (0.624)	Data 0.566 (0.566)	Loss 0.1497 (0.1497)	Prec 96.875% (96.875%)
Epoch: [61][100/391]	Time 0.084 (0.092)	Data 0.002 (0.007)	Loss 0.1940 (0.2018)	Prec 95.312% (93.062%)
Epoch: [61][200/391]	Time 0.087 (0.089)	Data 0.002 (0.006)	Loss 0.3139 (0.2112)	Prec 88.281% (92.615%)
Epoch: [61][300/391]	Time 0.090 (0.089)	Data 0.002 (0.004)	Loss 0.2901 (0.2163)	Prec 92.188% (92.455%)
Validation starts
Test: [0/79]	Time 0.499 (0.499)	Loss 0.2348 (0.2348)	Prec 88.281% (88.281%)
 * Prec 86.960% 
best acc: 87.060000
Epoch: [62][0/391]	Time 0.657 (0.657)	Data 0.590 (0.590)	Loss 0.1531 (0.1531)	Prec 92.969% (92.969%)
Epoch: [62][100/391]	Time 0.090 (0.093)	Data 0.002 (0.008)	Loss 0.2172 (0.2106)	Prec 91.406% (92.474%)
Epoch: [62][200/391]	Time 0.080 (0.089)	Data 0.002 (0.005)	Loss 0.2888 (0.2158)	Prec 88.281% (92.351%)
Epoch: [62][300/391]	Time 0.087 (0.089)	Data 0.002 (0.004)	Loss 0.1957 (0.2171)	Prec 93.750% (92.315%)
Validation s

Epoch: [76][100/391]	Time 0.086 (0.099)	Data 0.002 (0.008)	Loss 0.2847 (0.1874)	Prec 89.062% (93.332%)
Epoch: [76][200/391]	Time 0.089 (0.093)	Data 0.002 (0.005)	Loss 0.1251 (0.1868)	Prec 96.094% (93.311%)
Epoch: [76][300/391]	Time 0.095 (0.090)	Data 0.002 (0.004)	Loss 0.1478 (0.1913)	Prec 94.531% (93.233%)
Validation starts
Test: [0/79]	Time 0.421 (0.421)	Loss 0.3073 (0.3073)	Prec 87.500% (87.500%)
 * Prec 87.670% 
best acc: 87.950000
Epoch: [77][0/391]	Time 0.682 (0.682)	Data 0.615 (0.615)	Loss 0.2127 (0.2127)	Prec 92.188% (92.188%)
Epoch: [77][100/391]	Time 0.091 (0.093)	Data 0.002 (0.010)	Loss 0.2661 (0.1913)	Prec 92.969% (93.162%)
Epoch: [77][200/391]	Time 0.076 (0.090)	Data 0.002 (0.006)	Loss 0.2522 (0.2021)	Prec 89.062% (92.767%)
Epoch: [77][300/391]	Time 0.094 (0.089)	Data 0.002 (0.005)	Loss 0.2321 (0.2036)	Prec 91.406% (92.707%)
Validation starts
Test: [0/79]	Time 0.544 (0.544)	Loss 0.2807 (0.2807)	Prec 89.844% (89.844%)
 * Prec 85.960% 
best acc: 87.950000
Epoch: [78][0/391]	

Epoch: [91][200/391]	Time 0.089 (0.092)	Data 0.002 (0.006)	Loss 0.1140 (0.0775)	Prec 96.875% (97.388%)
Epoch: [91][300/391]	Time 0.090 (0.090)	Data 0.002 (0.005)	Loss 0.0569 (0.0754)	Prec 97.656% (97.501%)
Validation starts
Test: [0/79]	Time 0.446 (0.446)	Loss 0.1526 (0.1526)	Prec 95.312% (95.312%)
 * Prec 90.490% 
best acc: 90.530000
Epoch: [92][0/391]	Time 0.569 (0.569)	Data 0.503 (0.503)	Loss 0.1323 (0.1323)	Prec 95.312% (95.312%)
Epoch: [92][100/391]	Time 0.087 (0.092)	Data 0.002 (0.007)	Loss 0.0739 (0.0696)	Prec 96.875% (97.703%)
Epoch: [92][200/391]	Time 0.085 (0.090)	Data 0.002 (0.004)	Loss 0.1183 (0.0729)	Prec 93.750% (97.512%)
Epoch: [92][300/391]	Time 0.128 (0.089)	Data 0.086 (0.004)	Loss 0.0859 (0.0729)	Prec 96.875% (97.508%)
Validation starts
Test: [0/79]	Time 0.499 (0.499)	Loss 0.1693 (0.1693)	Prec 95.312% (95.312%)
 * Prec 90.270% 
best acc: 90.530000
Epoch: [93][0/391]	Time 0.695 (0.695)	Data 0.647 (0.647)	Loss 0.0644 (0.0644)	Prec 98.438% (98.438%)
Epoch: [93][100/391]	

In [ ]:
# HW

#  1. Train with 4 bits for both weight and activation to achieve >90% accuracy
#  2. Find x_int and w_int for the 2nd convolution layer
#  3. Check the recovered psum has similar value to the un-quantized original psum
#     (such as example 1 in W3S2)

In [5]:
#PATH = "result/VGG16_quant/model_best.pth.tar"
PATH = "result/resnet20_quant_2bit/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9053/10000 (91%)



In [6]:
#send an input and grap the value by using prehook like HW3
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()

for layer in model.modules():
    if isinstance(layer, torch.nn.Conv2d):
        print("prehooked")
        layer.register_forward_pre_hook(save_output)       ## Input for the module will be grapped       
####################################################

dataiter = iter(trainloader)
images, labels = dataiter.next()
images = images.to(device)
out = model(images)
print("1st convolution's input size:", save_output.outputs[0][0].size())
print("2nd convolution's input size:", save_output.outputs[1][0].size())

prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
1st convolution's input size: torch.Size([128, 3, 32, 32])
2nd convolution's input size: torch.Size([128, 16, 32, 32])


In [7]:
conv2= model.layer1[0].conv1
#print(conv2.weight_q)
#print(conv2.weight_quant.wgt_alpha.data.item())
#print(conv2.weight_quant.wgt_alpha.data.item()/ (2**4 - 1))

In [8]:
w_bit = 2
#weight_q = model.features[3].weight_q # quantized value is stored during the training

weight_q = conv2.weight_q
w_alpha = conv2.weight_quant.wgt_alpha.data.item()
w_delta = w_alpha / (2**(w_bit - 1) - 1)
weight_int = weight_q / w_delta
print(weight_int) # you should see clean integer numbers

tensor([[[[-0.0000,  0.1429, -1.0000],
          [ 0.5714,  1.0000, -1.0000],
          [-0.0000,  0.2857, -0.2857]],

         [[-0.2857,  0.5714,  0.4286],
          [-0.7143, -0.1429,  0.2857],
          [-0.4286, -0.7143,  0.1429]],

         [[ 1.0000, -0.1429, -0.1429],
          [ 0.8571, -0.4286, -0.4286],
          [-0.0000, -0.4286, -0.2857]],

         ...,

         [[-0.5714, -0.1429, -0.2857],
          [-0.8571,  1.0000, -0.4286],
          [-0.1429, -0.1429, -0.7143]],

         [[ 0.4286,  0.1429,  0.0000],
          [ 0.2857, -0.1429, -0.1429],
          [ 0.1429, -0.1429, -0.7143]],

         [[ 0.1429,  0.1429,  0.4286],
          [ 0.1429,  0.0000,  0.1429],
          [-0.1429, -0.2857,  0.0000]]],


        [[[ 0.1429,  0.8571,  1.0000],
          [-0.2857,  0.2857,  0.8571],
          [-0.8571, -0.2857,  0.1429]],

         [[-0.0000,  0.1429,  0.1429],
          [ 0.0000, -0.0000, -0.2857],
          [ 0.2857, -0.1429, -0.2857]],

         [[-0.1429, -1.0000, -0

In [ ]:
#print(save_output.outputs[1][0])
#print(conv2.act_alpha.data.item())


In [9]:
x_bit = 2
x =  save_output.outputs[1][0] # input of the 2nd conv layer
x_alpha  = conv2.act_alpha.data.item()
x_delta = x_alpha / (2**x_bit - 1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q / x_delta
print(x_int) # you should see clean integer numbers 

tensor([[[[1., 1., 0.,  ..., 1., 1., 1.],
          [1., 0., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [0., 1., 1.,  ..., 1., 1., 2.],
          [0., 1., 1.,  ..., 1., 1., 2.],
          [0., 1., 1.,  ..., 1., 1., 2.]],

         [[1., 1., 0.,  ..., 0., 1., 1.],
          [1., 1., 1.,  ..., 0., 0., 1.],
          [1., 1., 1.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [2., 2., 2.,  ..., 2., 2., 0.]],

         [[1., 0., 1.,  ..., 1., 1., 1.],
          [1., 0., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [0., 1., 1.,  ..., 1., 1., 3.],
          [0., 1., 1.,  ..., 1., 1., 3.],
          [1., 2., 2.,  ..., 2., 2., 3.]],

         ...,

         [[2., 1., 1.,  ..., 1., 1., 2.],
          [2., 1., 1.,  ..., 1., 1., 1.],
          [2., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 

In [10]:
conv_int = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)

output_int =  conv_int(x_int)
output_recovered = output_int * x_delta * w_delta
print(output_recovered)

tensor([[[[-2.1464e+01, -3.1994e+01, -1.8629e+01,  ..., -1.3769e+01,
           -1.2150e+01, -6.0748e+00],
          [-1.6199e+01, -2.1464e+01, -8.9096e+00,  ..., -1.2554e+01,
           -1.1340e+01, -8.9096e+00],
          [-2.1869e+01, -1.6199e+01, -2.8349e+00,  ..., -9.7196e+00,
           -1.2959e+01, -1.7009e+01],
          ...,
          [ 1.0983e-06, -1.6199e+00,  4.0498e-01,  ..., -1.2959e+01,
           -7.2897e+00, -1.3364e+01],
          [-1.8629e+01, -1.4984e+01, -1.6604e+01,  ..., -1.5389e+01,
           -1.3364e+01, -1.6604e+01],
          [-4.4548e+00, -4.0498e+00, -4.0498e+00,  ..., -2.8349e+00,
           -2.8349e+00, -1.6604e+01]],

         [[-2.4299e+00, -6.8847e+00, -4.4548e+00,  ..., -6.8847e+00,
           -6.8847e+00, -4.8598e+00],
          [-1.6199e+00, -5.2648e+00, -4.8598e+00,  ..., -5.2648e+00,
           -6.8847e+00, -4.4548e+00],
          [-3.2399e+00, -4.8598e+00, -7.6947e+00,  ..., -6.4797e+00,
           -7.2897e+00, -1.0125e+01],
          ...,
     

In [ ]:
#### input floating number / weight quantized version

#conv_ref = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
#conv_ref.weight = torch.nn.parameter.Parameter(weight_int)

#output_ref = conv_ref(x)
#print(output_ref)

In [11]:
#### input floating number / weight floating number version

conv_ref = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
#weight = model.features[3].weight
#mean = weight.data.mean()
#std = weight.data.std()
#conv_ref.weight = torch.nn.parameter.Parameter(weight.add(-mean).div(std))
conv_ref.weight = torch.nn.parameter.Parameter(weight_q)
output_ref = conv_ref(x)
print(output_ref)


tensor([[[[-2.4125e+01, -2.4126e+01, -2.0180e+01,  ..., -1.5358e+01,
           -1.0014e+01, -5.1404e+00],
          [-2.5405e+01, -1.4761e+01, -1.0680e+01,  ..., -8.6709e+00,
           -1.1456e+01, -1.0104e+01],
          [-2.4110e+01, -1.0237e+01, -6.3433e+00,  ..., -1.1045e+01,
           -1.6476e+01, -1.2275e+01],
          ...,
          [-4.1727e+00, -4.0139e+00, -5.6855e+00,  ..., -1.3655e+01,
           -5.7615e+00, -1.1215e+01],
          [-2.4244e+01, -2.1316e+01, -2.1341e+01,  ..., -1.5370e+01,
           -1.4013e+01, -2.3980e+01],
          [-4.0870e+00, -3.6631e+00, -3.5516e+00,  ..., -3.3250e+00,
           -3.6847e+00, -1.6156e+01]],

         [[-1.0068e+00, -3.5434e+00, -4.6018e+00,  ..., -3.1536e+00,
           -2.7975e+00, -2.3760e+00],
          [-2.6514e+00, -3.4339e+00, -3.8551e+00,  ..., -4.8918e+00,
           -4.3300e+00, -6.2175e+00],
          [-3.4571e+00, -2.5807e+00, -3.0288e+00,  ..., -7.5374e+00,
           -6.4359e+00, -9.2570e+00],
          ...,
     

In [12]:
difference = abs( output_ref - output_recovered )
print(difference.mean()) 

tensor(2.9977, device='cuda:0', grad_fn=<MeanBackward0>)
